In [2]:
#import libraries

import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [3]:
#import data, set display options

original_ride_data = pd.read_excel("uber_ride_data.xlsx")


display(original_ride_data)

,DATE,MONTH,YEAR,DRIVER,GENDER,TYPE,CITY,PRICE,PAYMENT,START TIME,END TIME,HOUR,CAR,MILES,TRIP TIME
0,2011-06-16,6,2011,Faris,m,-,San Francisco,NaN,NaN,21:57:00,NaN,21,NaN,NaN,NaN
1,2011-07-03,7,2011,Cal,m,-,San Francisco,19.00,NaN,01:13:00,01:26:00,1,NaN,NaN,NaN
2,2011-07-10,7,2011,Kader,m,-,San Francisco,NaN,NaN,23:49:00,NaN,23,NaN,NaN,NaN
3,2011-07-11,7,2011,Mohammad,m,-,San Francisco,65.00,NaN,05:36:00,06:08:00,5,NaN,NaN,NaN
4,2011-07-18,7,2011,Kamal,m,-,San Francisco,65.00,NaN,12:10:00,12:41:00,12,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,2016-12-22,12,2016,CLAUDION,m,POOL,San Francisco,4.75,6700.0,08:10:00,08:32:00,8,POOL,2.41,00:20:12
988,2016-12-22,12,2016,DEXTER,m,POOL,San Francisco,4.75,6700.0,18:18:00,18:43:00,18,POOL,1.28,00:21:03
989,2016-12-22,12,2016,Joseph,m,POOL,San Francisco,4.75,6700.0,14:37:00,14:49:00,14,POOL,0.57,00:08:03
990,2016-12-23,12,2016,Navpreet,m,uberXL,San Francisco,44.23,6700.0,16:08:00,17:03:00,16,uberXL,17.48,00:35:18


In [4]:
#clean data



#create new dataframe
df = original_ride_data


#lowercase column headers
df.columns = df.columns.str.lower()

#drop un-needed columns, drop "yellow" ride type in "type" column - cant identify what type of uber it is
df.drop(df[df['driver'] == 'UberEATS'].index,inplace = True)
df.drop(df[df['type'] == 'Yellow'].index,inplace = True)
df = df.drop(['gender', 'driver', 'payment', 'hour', 'car', 'trip time'], axis = 1)

#changing values in month column to month name 
df['month'] = df['date'].dt.strftime('%b')


#Insert N/A for missing values, and fix spelling variations of uber rides in "type" column ( example: UberBlack, Black)  
df = df.replace("-", np.nan)
df['type'] = df['type'].replace(['POOL', 'BLACK', 'SUV', 'SELECT', 'Black','uberCAB','TAXI','UberSELECT'], ['uberpool', 'uberblack', 'ubersuv',  'uberselect', 'uberblack', 'ubertaxi', 'ubertaxi','uberselect'])



#lowercase types of car rides
df['type'] = df['type'].str.lower()

#combining start and end columns with date column to meet datetime function requirements
df['start time'] = df['date'].dt.strftime('%Y-%m-%d').astype('str')+" "+df['start time'].astype('str')
df['end time'] = df['date'].dt.strftime('%Y-%m-%d').astype('str')+" "+df['end time'].astype('str')


#converting start and end time column to datetime
df['start time'] = pd.to_datetime(df['start time'], format = "%Y/%m/%d %H:%M:%S")
df['end time'] = pd.to_datetime(df['end time'], format = "%Y/%m/%d %H:%M:%S", errors ='coerce')

#modifying date of 'END TIME' values for rides near midnight that went into next day ( was resulting in travel time such as(-1 day 00:25:00))
df.loc[47,'end time'] = pd.to_datetime('2013-02-16 00:25:00')
df.loc[93,'end time'] = pd.to_datetime('2013-10-27 00:05:00')
df.loc[268,'end time'] = pd.to_datetime('2014-09-14 00:09:00')
df.loc[424,'end time'] = pd.to_datetime('2015-04-22 00:12:00')
df.loc[428,'end time'] = pd.to_datetime('2015-04-23 00:07:00')
df.loc[443,'end time'] = pd.to_datetime('2015-04-25 00:10:00')
df.loc[691,'end time'] = pd.to_datetime('2016-07-09 00:16:00')
df.loc[740,'end time'] = pd.to_datetime('2016-08-27 00:32:00')


#caculate difference between end time & start time to get accurate net travel time for each ride
travel_time = df['end time'] - df['start time']

#insert travel time column into dataframe 
df.insert(8, 'travel time', travel_time)


display(df)


,date,month,year,type,city,price,start time,end time,travel time,miles
0,2011-06-16,Jun,2011,NaN,San Francisco,NaN,2011-06-16 21:57:00,NaT,NaT,NaN
1,2011-07-03,Jul,2011,NaN,San Francisco,19.00,2011-07-03 01:13:00,2011-07-03 01:26:00,00:13:00,NaN
2,2011-07-10,Jul,2011,NaN,San Francisco,NaN,2011-07-10 23:49:00,NaT,NaT,NaN
3,2011-07-11,Jul,2011,NaN,San Francisco,65.00,2011-07-11 05:36:00,2011-07-11 06:08:00,00:32:00,NaN
4,2011-07-18,Jul,2011,NaN,San Francisco,65.00,2011-07-18 12:10:00,2011-07-18 12:41:00,00:31:00,NaN
...,...,...,...,...,...,...,...,...,...,...
985,2016-12-21,Dec,2016,uberpool,San Francisco,39.49,2016-12-21 15:56:00,2016-12-21 17:28:00,01:32:00,43.23
987,2016-12-22,Dec,2016,uberpool,San Francisco,4.75,2016-12-22 08:10:00,2016-12-22 08:32:00,00:22:00,2.41
988,2016-12-22,Dec,2016,uberpool,San Francisco,4.75,2016-12-22 18:18:00,2016-12-22 18:43:00,00:25:00,1.28
989,2016-12-22,Dec,2016,uberpool,San Francisco,4.75,2016-12-22 14:37:00,2016-12-22 14:49:00,00:12:00,0.57


In [5]:
# caculating Uber rides total milage  

#Would milage traveled be more than what one could have on a lease 
#caculating yearly total of milage
yearly_milage = pd.pivot_table( df, values = 'miles',index ='year', margins='True', aggfunc=sum,margins_name = 'Total Milage')
yearly_milage=np.round(yearly_milage, 0)
display(yearly_milage)



,miles
year,
2011,0.0
2012,3.0
2013,222.0
2014,1053.0
2015,800.0
2016,1197.0
Total Milage,3276.0


In [6]:
#caculating yearly cost of uber rides each year

yearly_cost = df[['year','price']]
yearly_cost =yearly_cost.groupby(['year']).sum().reset_index()
yearly_cost.columns = ['year', 'cost per year']
display(yearly_cost)

,year,cost per year
0,2011,262.00
1,2012,738.76
2,2013,1373.45
3,2014,3724.47
4,2015,3044.03
5,2016,2898.94


In [9]:








# setting up dataframe to display lease costs
car_list=[
 ('Bmw M3','8333', '1300', '500', '1500', '150'),
('Jeep Cherokee', '0', '1800','500', '1500', '150'),
('Leased Tesla model 3', '0', '500', '3000', '134','400')]
yearly_cost=pd.DataFrame(car_list, columns = ['car','depr', 'maintenance',  'insurance', 'gas/electric', 'registration'])
display(yearly_cost)



,car,depr,maintenance,insurance,gas/electric,registration
0,Bmw M3,8333,1300,500,1500,150
1,Jeep Cherokee,0,1800,500,1500,150
2,Leased Tesla model 3,0,500,3000,134,400
